# Workshop Azure Databricks
## 07. Classification Evaluation
<img src="https://raw.githubusercontent.com/retkowsky/images/master/AzureDatabricksLogo.jpg"><br>

# Documentation
Présentation https://azure.microsoft.com/fr-fr/services/databricks/

Documentation Azure Databricks : https://docs.microsoft.com/fr-fr/azure/databricks/

Documentation Azure ML : https://docs.microsoft.com/en-us/azure/machine-learning/

Github : https://github.com/Azure/MachineLearningNotebooks/tree/master/how-to-use-azureml/azure-databricks

## Evaluating a Classification Model

In this exercise, you will create a pipeline for a classification model, and then apply commonly used metrics to evaluate the resulting classifier.

### Prepare the Data

First, import the libraries you will need and prepare the training and test data:

In [0]:
import datetime
now = datetime.datetime.now()
print(now)

2021-02-10 13:55:07.223757

In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import VectorAssembler, StringIndexer, MinMaxScaler
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Load the source data
# File location and type
file_location = "/FileStore/tables/flights.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
data = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(data)

# Split the data
splits = data.randomSplit([0.7, 0.3])

train = splits[0]
test = splits[1]

DayofMonth,DayOfWeek,Carrier,OriginAirportID,DestAirportID,DepDelay,ArrDelay,Late
21,2,WN,10721,13342,26,57,1
13,1,AA,15016,12892,51,27,1
5,5,FL,10397,11433,9,4,0
22,1,US,11278,14100,35,71,1
23,4,WN,12451,10693,9,5,0
5,7,AA,11298,15016,39,42,1
4,6,UA,13930,14307,71,58,1
10,3,9E,14307,11433,68,140,1
29,7,UA,14057,14771,130,125,1
14,7,UA,14771,11292,20,42,1


### Define the Pipeline and Train the Model
Now define a pipeline that creates a feature vector and trains a classification model

In [0]:
monthdayIndexer = StringIndexer(inputCol="DayofMonth", outputCol="DayofMonthIdx")
weekdayIndexer = StringIndexer(inputCol="DayOfWeek", outputCol="DayOfWeekIdx")
carrierIndexer = StringIndexer(inputCol="Carrier", outputCol="CarrierIdx")
originIndexer = StringIndexer(inputCol="OriginAirportID", outputCol="OriginAirportIdx")
destIndexer = StringIndexer(inputCol="DestAirportID", outputCol="DestAirportIdx")
numVect = VectorAssembler(inputCols = ["DepDelay"], outputCol="numFeatures")
minMax = MinMaxScaler(inputCol = numVect.getOutputCol(), outputCol="normNums")
featVect = VectorAssembler(inputCols=["DayofMonthIdx", "DayOfWeekIdx", "CarrierIdx", "OriginAirportIdx", "DestAirportIdx", "normNums"], outputCol="features")

In [0]:
lr = LogisticRegression(labelCol="Late", featuresCol="features")

In [0]:
pipeline = Pipeline(stages=[monthdayIndexer, weekdayIndexer, carrierIndexer, originIndexer, destIndexer, numVect, minMax, featVect, lr])
model = pipeline.fit(train)

### Test the Model
Now you're ready to apply the model to the test data.

In [0]:
prediction = model.transform(test)
predicted = prediction.select("features", col("prediction").cast("Int"), col("Late").alias("trueLabel"))

predicted.show(100, truncate=False)

+---------------------------------------------+----------+---------+
features |prediction|trueLabel|
+---------------------------------------------+----------+---------+
[25.0,2.0,10.0,1.0,25.0,0.7263681592039801] |1 |1 |
[25.0,2.0,10.0,11.0,41.0,0.23383084577114427]|0 |0 |
[25.0,2.0,10.0,11.0,12.0,0.38308457711442784]|1 |1 |
[25.0,2.0,10.0,11.0,12.0,0.4427860696517413] |1 |1 |
[25.0,2.0,10.0,48.0,12.0,0.736318407960199] |1 |1 |
[25.0,2.0,10.0,48.0,16.0,0.2537313432835821] |0 |0 |
[25.0,2.0,10.0,38.0,1.0,0.2288557213930348] |0 |0 |
[25.0,2.0,10.0,14.0,32.0,0.24378109452736318]|0 |0 |
[25.0,2.0,10.0,14.0,11.0,0.263681592039801] |0 |1 |
[25.0,2.0,10.0,49.0,15.0,0.2288557213930348] |0 |1 |
[25.0,2.0,10.0,49.0,33.0,0.1990049751243781] |0 |0 |
[25.0,2.0,10.0,12.0,19.0,0.47761194029850745]|1 |1 |
[25.0,2.0,10.0,12.0,22.0,0.2537313432835821] |0 |1 |
[25.0,2.0,10.0,12.0,15.0,0.2537313432835821] |0 |0 |
[25.0,2.0,10.0,12.0,25.0,0.900497512437811] |1 |1 |
[25.0,2.0,10.0,16.0,15.0,0.23880597014925373]|0 |0 |
[25.0,2.0,10.0,47.0,32.0,0.2288557213930348] |0 |0 |
[25.0,2.0,10.0,17.0,39.0,0.582089552238806] |1 |1 |
[25.0,2.0,10.0,54.0,15.0,0.6368159203980099] |1 |1 |
[25.0,2.0,10.0,0.0,41.0,0.472636815920398] |1 |1 |
[25.0,2.0,10.0,0.0,16.0,0.5522388059701493] |1 |1 |
[25.0,2.0,10.0,64.0,12.0,0.5671641791044776] |1 |1 |
[25.0,2.0,10.0,67.0,15.0,0.26865671641791045]|0 |0 |
[25.0,2.0,10.0,29.0,33.0,0.21393034825870647]|0 |0 |
[25.0,2.0,2.0,1.0,24.0,0.4427860696517413] |1 |1 |
[25.0,2.0,2.0,34.0,5.0,0.34328358208955223] |1 |0 |
[25.0,2.0,2.0,27.0,2.0,0.2537313432835821] |0 |1 |
[25.0,2.0,2.0,11.0,24.0,0.21890547263681592] |0 |0 |
[25.0,2.0,2.0,11.0,24.0,0.34825870646766166] |1 |1 |
[25.0,2.0,2.0,11.0,24.0,0.4626865671641791] |1 |1 |
[25.0,2.0,2.0,3.0,48.0,0.2885572139303483] |0 |0 |
[25.0,2.0,2.0,3.0,32.0,0.24378109452736318] |0 |0 |
[25.0,2.0,2.0,3.0,9.0,0.24875621890547264] |0 |0 |
[25.0,2.0,2.0,3.0,9.0,0.25870646766169153] |0 |0 |
[25.0,2.0,2.0,3.0,22.0,0.24875621890547264] |0 |0 |
[25.0,2.0,2.0,3.0,11.0,0.29850746268656714] |0 |0 |
[25.0,2.0,2.0,3.0,8.0,0.24378109452736318] |0 |0 |
[25.0,2.0,2.0,3.0,2.0,0.24875621890547264] |0 |0 |
[25.0,2.0,2.0,3.0,2.0,0.263681592039801] |0 |0 |
[25.0,2.0,2.0,3.0,13.0,0.24875621890547264] |0 |0 |
[25.0,2.0,2.0,3.0,14.0,0.25870646766169153] |0 |0 |
[25.0,2.0,2.0,3.0,52.0,0.26865671641791045] |0 |0 |
[25.0,2.0,2.0,3.0,59.0,0.2736318407960199] |0 |0 |
[25.0,2.0,2.0,3.0,33.0,0.6019900497512438] |1 |1 |
[25.0,2.0,2.0,3.0,39.0,0.5422885572139303] |1 |1 |
[25.0,2.0,2.0,14.0,5.0,0.681592039800995] |1 |1 |
[25.0,2.0,2.0,12.0,24.0,0.2537313432835821] |0 |0 |
[25.0,2.0,2.0,12.0,3.0,0.5771144278606966] |1 |1 |
[25.0,2.0,2.0,7.0,5.0,0.36318407960199006] |1 |0 |
[25.0,2.0,2.0,2.0,5.0,0.5074626865671642] |1 |1 |
[25.0,2.0,2.0,2.0,24.0,0.9154228855721392] |1 |1 |
[25.0,2.0,2.0,2.0,1.0,0.26865671641791045] |0 |0 |
[25.0,2.0,2.0,13.0,24.0,0.48756218905472637] |1 |1 |
[25.0,2.0,2.0,13.0,1.0,0.5671641791044776] |1 |1 |
[25.0,2.0,2.0,21.0,7.0,0.24378109452736318] |0 |0 |
[25.0,2.0,2.0,21.0,2.0,0.6268656716417911] |1 |1 |
[25.0,2.0,2.0,21.0,13.0,0.582089552238806] |1 |1 |
[25.0,2.0,2.0,21.0,14.0,0.24378109452736318] |0 |0 |
[25.0,2.0,2.0,21.0,14.0,0.4925373134328358] |1 |1 |
[25.0,2.0,2.0,21.0,28.0,0.43781094527363185] |1 |1 |
[25.0,2.0,2.0,36.0,24.0,0.24875621890547264] |0 |1 |
[25.0,2.0,2.0,66.0,2.0,0.2935323383084577] |0 |1 |
[25.0,2.0,2.0,54.0,5.0,0.21393034825870647] |0 |0 |
[25.0,2.0,2.0,0.0,6.0,0.527363184079602] |1 |1 |
[25.0,2.0,2.0,0.0,56.0,0.5223880597014925] |1 |1 |
[25.0,2.0,2.0,0.0,29.0,0.39303482587064675] |1 |1 |
[25.0,2.0,2.0,63.0,1.0,0.8109452736318408] |1 |1 |
[25.0,2.0,2.0,42.0,5.0,0.21890547263681592] |0 |0 |
[25.0,2.0,2.0,5.0,12.0,0.24378109452736318] |0 |1 |
[25.0,2.0,2.0,5.0,1.0,0.3383084577114428] |1 |1 |
[25.0,2.0,12.0,7.0,34.0,0.21890547263681592] |0 |0 |
[25.0,2.0,12.0,7.0,34.0,0.2288557213930348] |0 |0 |
[25.0,2.0,12.0,7.0,34.0,0.835820895522388] |1 |1 |
[25.0,2.0,12.0,2.0,34.0,0.2537313432835821] |

### Compute Confusion Matrix Metrics
Classifiers are typically evaluated by creating a *confusion matrix*, which indicates the number of:
- True Positives
- True Negatives
- False Positives
- False Negatives

From these core measures, other evaluation metrics such as *precision* and *recall* can be calculated.

In [0]:
tp = float(predicted.filter("prediction == 1.0 AND truelabel == 1").count())
fp = float(predicted.filter("prediction == 1.0 AND truelabel == 0").count())
tn = float(predicted.filter("prediction == 0.0 AND truelabel == 0").count())
fn = float(predicted.filter("prediction == 0.0 AND truelabel == 1").count())

metrics = spark.createDataFrame([
 ("TP", tp),
 ("FP", fp),
 ("TN", tn),
 ("FN", fn),
 ("Precision", tp / (tp + fp)),
 ("Recall", tp / (tp + fn))],["metric", "value"])


In [0]:
display(metrics)

metric,value
TP,89364.0
FP,7348.0
TN,82025.0
FN,13059.0
Precision,0.9240218380345769
Recall,0.8724993409683371


### View the Raw Prediction and Probability
The prediction is based on a raw prediction score that describes a labelled point in a logistic function. This raw prediction is then converted to a predicted label of 0 or 1 based on a probability vector that indicates the confidence for each possible label value (in this case, 0 and 1). The value with the highest confidence is selected as the prediction.

In [0]:
prediction.select("rawPrediction", "probability", "prediction", col("Late").alias("trueLabel")).show(100, truncate=False)

+------------------------------------------+------------------------------------------+----------+---------+
rawPrediction |probability |prediction|trueLabel|
+------------------------------------------+------------------------------------------+----------+---------+
[-10.241147330854472,10.241147330854472] |[3.567062624501566E-5,0.9999643293737549] |1.0 |1 |
[2.5525864553174458,-2.5525864553174458] |[0.9277470829616334,0.07225291703836653] |0.0 |0 |
[-1.5562768259603015,1.5562768259603015] |[0.1741815475760053,0.8258184524239948] |1.0 |1 |
[-3.08022386949445,3.08022386949445] |[0.04393041179849424,0.9560695882015058] |1.0 |1 |
[-10.364995574854328,10.364995574854328] |[3.1515625390585366E-5,0.9999684843746093]|1.0 |1 |
[1.9948171427858838,-1.9948171427858838] |[0.880251836338757,0.11974816366124291] |0.0 |0 |
[2.4189345961780555,-2.4189345961780555] |[0.9182598123078335,0.08174018769216654] |0.0 |0 |
[2.222665785199479,-2.222665785199479] |[0.9022665212423462,0.09773347875765383] |0.0 |0 |
[1.4981693297240763,-1.4981693297240763] |[0.8173012794638596,0.1826987205361404] |0.0 |1 |
[2.6251056367295,-2.6251056367295] |[0.9324599619945309,0.06754003800546907] |0.0 |1 |
[3.5726626793228755,-3.5726626793228755] |[0.9726860200251619,0.02731397997483813] |0.0 |0 |
[-3.8914008542732947,3.8914008542732947] |[0.020008222754603348,0.9799917772453967] |1.0 |1 |
[1.8543311457841467,-1.8543311457841467] |[0.8646348275901574,0.13536517240984258] |0.0 |1 |
[1.782159776573918,-1.782159776573918] |[0.8559633497350579,0.1440366502649421] |0.0 |0 |
[-14.62416457236475,14.62416457236475] |[4.454570434923169E-7,0.9999995545429564] |1.0 |1 |
[2.1856295564141783,-2.1856295564141783] |[0.898951597362783,0.10104840263721707] |0.0 |0 |
[2.7891374524759787,-2.7891374524759787] |[0.9420860021006294,0.05791399789937068] |0.0 |0 |
[-6.324000494732928,6.324000494732928] |[0.0017895489987391328,0.9982104510012608]|1.0 |1 |
[-7.760428720391282,7.760428720391282] |[4.260921573210609E-4,0.999573907842679] |1.0 |1 |
[-3.6050300209501387,3.6050300209501387] |[0.026467078011496725,0.9735329219885034] |1.0 |1 |
[-5.894714302365534,5.894714302365534] |[0.0027463995485799936,0.9972536004514201]|1.0 |1 |
[-5.957213542347352,5.957213542347352] |[0.0025804349413214023,0.9974195650586787]|1.0 |1 |
[1.7103145263453179,-1.7103145263453179] |[0.8468770752959887,0.15312292470401134] |0.0 |0 |
[3.0792608236557246,-3.0792608236557246] |[0.9560291219948993,0.04397087800510075] |0.0 |0 |
[-2.583719033062499,2.583719033062499] |[0.07019361439689778,0.9298063856031022] |1.0 |1 |
[-0.05421610387371878,0.05421610387371878]|[0.48644929309872853,0.5135507069012715] |1.0 |0 |
[2.161428591448856,-2.161428591448856] |[0.8967319165185532,0.10326808348144677] |0.0 |1 |
[3.1872899275823654,-3.1872899275823654] |[0.9603531631725176,0.03964683682748235] |0.0 |0 |
[-0.11459533340828898,0.11459533340828898]|[0.4713824771205872,0.5286175228794129] |1.0 |1 |
[-3.0354938335154085,3.0354938335154085] |[0.045847894009443745,0.9541521059905562] |1.0 |1 |
[1.6118303666474807,-1.6118303666474807] |[0.833665353701202,0.16633464629879793] |0.0 |0 |
[2.5898275196747145,-2.5898275196747145] |[0.9302040197551935,0.06979598024480638] |0.0 |0 |
[2.225697433879928,-2.225697433879928] |[0.9025335311814,0.09746646881860002] |0.0 |0 |
[1.9717062599575694,-1.9717062599575694] |[0.8777942642491174,0.12220573575088253] |0.0 |0 |
[2.3597299766989233,-2.3597299766989233] |[0.9137045169304783,0.08629548306952169] |0.0 |0 |
[0.9763619554710594,-0.9763619554710594] |[0.7263857526617695,0.2736142473382305] |0.0 |0 |
[2.3423828252396452,-2.3423828252396452] |[0.9123268665060992,0.0876731334939008] |0.0 |0 |
[2.153526064669699,-2.153526064669699] |[0.8959978148928246,0.10400218510717552] |0.0 |0 |
[1.7725393037861616,-1.7725393037861616] |[0.8547731741248146,0.1452268258751854] |0.0 |0 |
[2.266938216285772,-2.266938216285772] |[0.9061016103727286,0.09389838962727129] |0.0 |0 |
[2.0232572379648754,-2.0232572379648754] |[0.883

Note that the results include rows where the probability for 0 (the first value in the **probability** vector) is only slightly higher than the probability for 1 (the second value in the **probability** vector). The default *discrimination threshold* (the boundary that decides whether a probability is predicted as a 1 or a 0) is set to 0.5; so the prediction with the highest probability is always used, no matter how close to the threshold.

### Review the Area Under ROC
Another way to assess the performance of a classification model is to measure the area under a *received operator characteristic (ROC) curve* for the model. The **spark.ml** library includes a **BinaryClassificationEvaluator** class that you can use to compute this. A ROC curve plots the True Positive and False Positive rates for varying threshold values (the probability value over which a class label is predicted). The area under this curve gives an overall indication of the models accuracy as a value between 0 and 1. A value under 0.5 means that a binary classification model (which predicts one of two possible labels) is no better at predicting the right class than a random 50/50 guess.

In [0]:
evaluator = BinaryClassificationEvaluator(labelCol="Late", rawPredictionCol="rawPrediction", metricName="areaUnderROC")
auc = evaluator.evaluate(prediction)

print ("AUC = ", auc)

AUC = 0.9489970058752689

### Review the AreaUnderPR

In [0]:
evaluator = BinaryClassificationEvaluator(labelCol="Late", rawPredictionCol="rawPrediction", metricName="areaUnderPR")
areaUnderPR = evaluator.evaluate(prediction)
print ("Area Under PR = ", areaUnderPR)

Area Under PR = 0.9646811752032072

> You can open Lab08